# Valentin Lhermitte

In [1]:
import numpy as np
import scipy.io as sio

In [2]:
def load_data():
    mat = sio.loadmat('ocr_names.mat')
    trn_data = mat.get('TrnData')
    tst_data = mat.get('TstData')

    trn_size = trn_data.shape[1]
    trn_x = list()
    trn_y = list()
    for i in range(trn_size):
        trn_x.append(trn_data[0, i][1])
        trn_y.append(trn_data[0, i][2][0])

    tst_size = tst_data.shape[1]
    tst_x = list()
    tst_y = list()
    for i in range(tst_size):
        tst_x.append(tst_data[0, i][1])
        tst_y.append(tst_data[0, i][2][0])

    return trn_x, trn_y, tst_x, tst_y

In [3]:
trn_X, trn_Y, tst_X, tst_Y = load_data()

In [4]:
labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
          'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
          'u', 'v', 'w', 'x', 'y', 'z']

# 1. Assigment 1 : Independent linear multi-class classifier

In [5]:
class MultiClassPerceptron:
    def __init__(self, n_letters=26, n_features=8256):
        self.n_letters = n_letters
        self.n_features = n_features
        self.W = np.zeros(shape=(self.n_letters, self.n_features))
        self.b = np.ones(shape=(self.n_letters, 1))
        self.loss_sequence = 0
        self.loss_char = 0

    def forward(self, x):
        scores = np.dot(self.W, x) + self.b
        return np.argmax(scores, axis=0)

    def backward(self, x, y_corr, y_predicted):
        if not np.array_equal(y_corr, y_predicted):
            self.loss_sequence += 1
        for i in range(len(y_predicted)):
            if y_predicted[i] != y_corr[i]:
                self.loss_char += 1
                self.W[y_corr[i]] += x[:, i]
                self.b[y_corr[i]] += 1
                self.W[y_predicted[i]] -= x[:, i]
                self.b[y_predicted[i]] -= 1

    def train(self, X, Y, n_epochs=100):
        for epoch in range(n_epochs):
            print('Epoch: ', epoch)
            self.loss_char = 0
            self.loss_sequence = 0
            for i in range(len(X)):
                y_predicted = self.forward(X[i])
                y_corr = list(map(lambda c: labels.index(c), Y[i]))
                self.backward(X[i], y_corr, y_predicted)

            print('Loss char: ', self.loss_char / len(X))
            print('Loss sequence: ', self.loss_sequence / len(X))
            if self.loss_sequence == 0:
                break

    def predict(self, X):
        # Predict the label of image X
        return [self.forward(x) for x in X]

    def evaluate(self, X, Y):
        n_seq_errors = 0
        n_char_errors = 0
        n_letters = 0
        for i in range(len(X)):
            y_predicted = self.forward(X[i])
            y_corr = list(map(lambda c: labels.index(c), Y[i]))
            if not np.array_equal(y_corr, y_predicted):
                n_seq_errors += 1
                for j in range(len(y_predicted)):
                    if y_predicted[j] != y_corr[j]:
                        n_char_errors += 1
            n_letters += len(Y[i])
        R_char = n_char_errors / n_letters
        R_seq = n_seq_errors / len(X)
        return R_char, R_seq

In [6]:
# Train the model
model1 = MultiClassPerceptron(n_letters=26, n_features=8256)
model1.train(trn_X, trn_Y, n_epochs=100)

Epoch:  0
Loss char:  2.568
Loss sequence:  0.925
Epoch:  1
Loss char:  1.541
Loss sequence:  0.79
Epoch:  2
Loss char:  1.194
Loss sequence:  0.691
Epoch:  3
Loss char:  0.982
Loss sequence:  0.633
Epoch:  4
Loss char:  0.747
Loss sequence:  0.537
Epoch:  5
Loss char:  0.63
Loss sequence:  0.454
Epoch:  6
Loss char:  0.533
Loss sequence:  0.396
Epoch:  7
Loss char:  0.459
Loss sequence:  0.365
Epoch:  8
Loss char:  0.364
Loss sequence:  0.294
Epoch:  9
Loss char:  0.294
Loss sequence:  0.26
Epoch:  10
Loss char:  0.31
Loss sequence:  0.261
Epoch:  11
Loss char:  0.295
Loss sequence:  0.254
Epoch:  12
Loss char:  0.224
Loss sequence:  0.202
Epoch:  13
Loss char:  0.217
Loss sequence:  0.194
Epoch:  14
Loss char:  0.21
Loss sequence:  0.188
Epoch:  15
Loss char:  0.188
Loss sequence:  0.166
Epoch:  16
Loss char:  0.154
Loss sequence:  0.14
Epoch:  17
Loss char:  0.148
Loss sequence:  0.138
Epoch:  18
Loss char:  0.137
Loss sequence:  0.126
Epoch:  19
Loss char:  0.118
Loss sequence:  0.

In [9]:
# Evaluate the model
R_char_model1, R_seq_model1 = model1.evaluate(tst_X, tst_Y)
print('R_char: ', R_char_model1)
print('R_seq: ', R_seq_model1)

R_char:  0.2502360717658168
R_seq:  0.694


# 2. Assigment 2 :  Linear structured classifier modeling pair-wise dependency

In [10]:
class LinearStructuredClassifierPairwise:
    def __init__(self, n_labels=26, n_features=8256):
        self.n_labels = n_labels
        self.n_features = n_features
        self.W = np.zeros(shape=(self.n_labels, self.n_features))
        self.b = np.ones(shape=(self.n_labels, 1))
        self.g = np.zeros(shape=(self.n_labels, self.n_labels))
        self.loss_sequence = 0
        self.loss_char = 0

    def find_next_y(self, Q, F_list, y_pred, index):
        # Function using dynamic programming to find the best sequence of labels
        q_list = Q[:, index]
        next_F_list = np.zeros(self.n_labels)
        for i in range(self.n_labels):
            next_F_list[i] = q_list[i] + max(F_list + self.g[:, i])

        if index == len(y_pred) - 1:
            y_pred[index] = np.argmax(next_F_list)
        else:
            y_pred = self.find_next_y(Q, next_F_list, y_pred, index + 1)

        y_pred[index - 1] = np.argmax(F_list + self.g[:, y_pred[index]])
        return y_pred
    
    def forward(self, x):
        Q = np.dot(self.W, x) + self.b
        q_1 = Q[:, 0]

        y_pred = np.zeros(x.shape[1]).astype(np.int64)
        y_pred = self.find_next_y(Q, q_1, y_pred, 1)
        return y_pred
    

    def backward(self, x, y_corr, y_predicted):
        if not np.array_equal(y_corr, y_predicted):
            self.loss_sequence += 1
        for i in range(len(y_predicted)):
            if y_predicted[i] != y_corr[i]:
                self.loss_char += 1
                self.W[y_corr[i]] += x[:, i]
                self.b[y_corr[i]] += 1
                self.W[y_predicted[i]] -= x[:, i]
                self.b[y_predicted[i]] -= 1
            if i > 0:
                if y_predicted[i] != y_corr[i] and y_predicted[i - 1] == y_corr[i - 1]:
                    self.g[y_corr[i - 1], y_predicted[i]] -= 1
                    self.g[y_corr[i - 1], y_corr[i]] += 1
                elif y_predicted[i] == y_corr[i] and y_predicted[i - 1] != y_corr[i - 1]:
                    self.g[y_predicted[i - 1], y_corr[i]] -= 1
                    self.g[y_corr[i - 1], y_corr[i]] += 1
                elif y_predicted[i] != y_corr[i] and y_predicted[i - 1] != y_corr[i - 1]:
                    self.g[y_predicted[i - 1], y_predicted[i]] -= 1
                    self.g[y_corr[i - 1], y_corr[i]] += 1

    def train(self, X, Y, n_epochs=100):
        for epoch in range(n_epochs):
            print('Epoch: ', epoch)
            self.loss_char = 0
            self.loss_sequence = 0
            for i in range(len(X)):
                y_predicted = self.forward(X[i])
                y_corr = np.array(list(map(lambda c: labels.index(c), Y[i])))
                self.backward(X[i], y_corr, y_predicted)

            print('Loss char: ', self.loss_char / len(X))
            print('Loss sequence: ', self.loss_sequence / len(X))
            if self.loss_sequence == 0:
                break

    def predict(self, X):
        # Predict the label of image X
        return [self.forward(x) for x in X]

    def evaluate(self, X, Y):
        n_seq_errors = 0
        n_char_errors = 0
        n_letters = 0
        for i in range(len(X)):
            y_predicted = self.forward(X[i])
            y_corr = list(map(lambda c: labels.index(c), Y[i]))
            if not np.array_equal(y_corr, y_predicted):
                n_seq_errors += 1
                for j in range(len(y_predicted)):
                    if y_predicted[j] != y_corr[j]:
                        n_char_errors += 1
            n_letters += len(Y[i])
        R_char = n_char_errors / n_letters
        R_seq = n_seq_errors / len(X)
        return R_char, R_seq

In [11]:
model2 = LinearStructuredClassifierPairwise(n_labels=26, n_features=8256)
model2.train(trn_X, trn_Y)

Epoch:  0
Loss char:  1.934
Loss sequence:  0.616
Epoch:  1
Loss char:  0.66
Loss sequence:  0.279
Epoch:  2
Loss char:  0.447
Loss sequence:  0.211
Epoch:  3
Loss char:  0.333
Loss sequence:  0.162
Epoch:  4
Loss char:  0.249
Loss sequence:  0.132
Epoch:  5
Loss char:  0.178
Loss sequence:  0.092
Epoch:  6
Loss char:  0.108
Loss sequence:  0.063
Epoch:  7
Loss char:  0.119
Loss sequence:  0.062
Epoch:  8
Loss char:  0.122
Loss sequence:  0.068
Epoch:  9
Loss char:  0.079
Loss sequence:  0.046
Epoch:  10
Loss char:  0.081
Loss sequence:  0.049
Epoch:  11
Loss char:  0.03
Loss sequence:  0.022
Epoch:  12
Loss char:  0.065
Loss sequence:  0.038
Epoch:  13
Loss char:  0.097
Loss sequence:  0.05
Epoch:  14
Loss char:  0.043
Loss sequence:  0.024
Epoch:  15
Loss char:  0.029
Loss sequence:  0.015
Epoch:  16
Loss char:  0.01
Loss sequence:  0.008
Epoch:  17
Loss char:  0.02
Loss sequence:  0.012
Epoch:  18
Loss char:  0.002
Loss sequence:  0.002
Epoch:  19
Loss char:  0.002
Loss sequence:  0

In [12]:
# Evaluate the model
R_char_model2, R_seq_model2 = model2.evaluate(tst_X, tst_Y)
print('R_char: ', R_char_model2)
print('R_seq: ', R_seq_model2)

R_char:  0.061378659112370164
R_seq:  0.142


# 3. Assignment 3 : Linear structured classifier for fixed number of sequences

In [13]:
class SequenceLinearClassifier:
    def __init__(self, n_labels, n_features, sequences):
        self.n_classes = n_labels
        self.n_features = n_features
        self.W = np.zeros(shape=(n_labels, n_features))
        self.b = np.ones(shape=(n_labels, 1))
        self.sequences = sequences
        self.loss_sequence = 0
        self.loss_char = 0

    def forward(self, x):
        Q = np.dot(self.W, x) + self.b
        y_pred = np.zeros(x.shape[1])

        max_seq_score = float('-inf')
        max_seq_idx = 0
        sequences_l = self.sequences[x.shape[1]]
        for i in range(len(sequences_l)):
            sequence = sequences_l[i]
            letters = np.array([ord(letter) - 97 for letter in sequence])
            score = np.sum(Q[letters, range(len(sequence))])
            if score > max_seq_score:
                max_seq_score = score
                max_seq_idx = i
            y_pred = sequences_l[max_seq_idx]
        return y_pred

    def backward(self, x, y_corr, y_predicted):
        if not np.array_equal(y_corr, y_predicted):
            self.loss_sequence += 1
        for i in range(len(y_predicted)):
            if y_predicted[i] != y_corr[i]:
                self.loss_char += 1
                self.W[y_corr[i]] += x[:, i]
                self.b[y_corr[i]] += 1
                self.W[y_predicted[i]] -= x[:, i]
                self.b[y_predicted[i]] -= 1

    def train(self, x, y, epochs=100):
        for i in range(epochs):
            print('Epoch: ', i)
            self.loss_char = 0
            self.loss_sequence = 0
            for j in range(len(x)):
                y_corr = y[j]
                y_predicted = self.forward(x[j])
                # map letters to numbers
                y_corr = list(map(lambda c: ord(c) - 97, y_corr))
                y_predicted = list(map(lambda c: ord(c) - 97, y_predicted))
                self.backward(x[j], y_corr, y_predicted)

            print('Loss char: ', self.loss_char / len(x))
            print('Loss sequence: ', self.loss_sequence / len(x))
            if self.loss_sequence == 0:
                break

    def predict(self, x):
        return self.forward(x)

    def evaluate(self, X, Y):
        n_seq_errors = 0
        n_char_errors = 0
        n_letters = 0
        for i in range(len(X)):
            y_predicted = self.forward(X[i])
            y_corr = Y[i]
            if not np.array_equal(y_corr, y_predicted):
                n_seq_errors += 1
                for j in range(len(y_predicted)):
                    if y_predicted[j] != y_corr[j]:
                        n_char_errors += 1
            n_letters += len(Y[i])
        R_char = n_char_errors / n_letters
        R_seq = n_seq_errors / len(X)
        return R_char, R_seq

In [14]:
sequences = [[],
             [],
             ["bo", "ty"],
             ["max"],
             ["cruz", "drew", "greg", "hugh", "jack"],
             ["brock", "devyn", "elvis", "floyd", "quinn", "ralph", "steve", "tariq"],
             ["dwight", "joseph", "philip"],
             [],
             ["clifford"]]

In [15]:
model3 = SequenceLinearClassifier(n_labels=26, n_features=8256, sequences=sequences)
model3.train(trn_X, trn_Y)

Epoch:  0
Loss char:  0.752
Loss sequence:  0.182
Epoch:  1
Loss char:  0.139
Loss sequence:  0.033
Epoch:  2
Loss char:  0.011
Loss sequence:  0.003
Epoch:  3
Loss char:  0.0
Loss sequence:  0.0


In [16]:
R_char_model3, R_seq_model3 = model3.evaluate(tst_X, tst_Y)
print('R_char: ', R_char_model3)
print('R_seq: ', R_seq_model3)

R_char:  0.010387157695939566
R_seq:  0.014
